In [1]:
import pandas as pd
import numpy as np

In [2]:
tables=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)
df1=tables[0]
df2=df1[df1['Borough']!='Not assigned']
df2.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [3]:
df3=df2.groupby(['Postcode','Borough']).agg(','.join)

In [4]:
print(df3[df3['Neighbourhood']=='Not assigned'])

df4=df3.reset_index()
df4.loc[df4['Neighbourhood']=='Not assigned','Neighbourhood']=df4.loc[df4['Neighbourhood']=='Not assigned']['Borough']
df4[df4['Postcode']=='M7A']

                      Neighbourhood
Postcode Borough                   
M7A      Queen's Park  Not assigned


,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [5]:
df4.shape[0]

103

I was unable to use the Geocoder package, after mulitple attempts. Accordingly, I used the provided 'Geospatial_Coordinates.csv' file, which I read directly into a dataframe.

In [7]:
geodf=pd.read_csv("C:\\Users\\SAGHAR\\Desktop\\Geospatial_Coordinates.csv")
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
dfAll=pd.merge(df4,geodf,left_on='Postcode',right_on='Postal Code')
dfAll.drop(columns='Postal Code',inplace=True)
dfAll.rename(columns={'Postcode':'PostalCode'},inplace=True)
dfAll.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Create a map of toronto with neighbourhoods

In [9]:
import folium

tlat=43.6532
tlong=-79.3832
torontoMap=folium.Map(location=[tlat,tlong],zoom_start=10.7)

for lat,long,borough,neighborhood in zip(dfAll['Latitude'],dfAll['Longitude'],dfAll['Borough'],dfAll['Neighbourhood']):
    label='{}, {}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,long],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='#3186cc',
                       fill_opacity=0.7,
                       parse_html=False).add_to(torontoMap)
    
torontoMap

In [10]:
torontoData=dfAll.loc[dfAll.Borough.str.contains('Toronto')].reset_index(drop=True)
torontoData.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [11]:
print('The geographical coordinates of Toronto are {}, {}'.format(tlat,tlong))

The geographical coordinates of Toronto are 43.6532, -79.3832


Create a visualization of the Boroughs that have Toronto in the name

In [12]:
scarboroughMap=folium.Map(location=[tlat,tlong],zoom_start=12)
for lat,long,label in zip(torontoData['Latitude'],torontoData['Longitude'],torontoData['Neighbourhood']):
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,long],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='#3186cc',
                       fil_opacity=0.7,
                       parse_html=False).add_to(scarboroughMap)
scarboroughMap

Define Foursquare Credentials and Version

In [13]:
clientID='LYANKHBBTLDIQLFHBPC51N32RULMHYA2SZ3UALO34UO1UWVQ'
clientSecret='VAS21GKNZXMJOLAFPRDMMFTLYRWZXP4OTWQBWPDHMSOYLU4S'
version='20180605'
print('Your credentials:')
print('Client ID:  '+ clientID)
print('Client Secret:  '+clientSecret)

Your credentials:
Client ID:  LYANKHBBTLDIQLFHBPC51N32RULMHYA2SZ3UALO34UO1UWVQ
Client Secret:  VAS21GKNZXMJOLAFPRDMMFTLYRWZXP4OTWQBWPDHMSOYLU4S


In [14]:
nName=torontoData.loc[0,'Neighbourhood']
nLat=torontoData.loc[0,'Latitude']
nLong=torontoData.loc[0,'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(nName,nLat,nLong))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


Here we obtain the top 100 venues that are located within Toronto Boroughs with Toronto in the name within a radius of 500 meters

In [15]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(clientID,
                                                                                                                           clientSecret,
                                                                                                                           tlat,
                                                                                                                           tlong,
                                                                                                                           version,
                                                                                                                           radius,
                                                                                                                           LIMIT)

Here we send the get request and examine the results

In [16]:
import requests
import json
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ddb897683525f001be81ffc'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5227bb01498e17bf485e6202-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
          'suffix': '.png'},
         'id': '4f2a25ac4b909258e854f55f',
         'name': 'Neighborhood',
         'pluralName': 'Neighborhoods',
         'primary': True,
         'shortName': 'Neighborhood'}],
       'id': '5227bb01498e17bf485e6202',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 168,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng': -79.38529600606677}],
        'lat': 43.6532

Using the get_category_type function from the IBM Foursquare lab. The function extracts the category of the venue

In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe

In [18]:
from pandas.io.json import json_normalize
venues=results['response']['groups'][0]['items']
venuesNearBy=json_normalize(venues)

# filter columns
filtered_columns=['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venuesNearBy=venuesNearBy.loc[:, filtered_columns]

# clean columns
venuesNearBy.columns = [col.split(".")[-1] for col in venuesNearBy.columns]

venuesNearBy.head()

,name,categories,lat,lng
0,Downtown Toronto,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.653232,-79.385296
1,Nathan Phillips Square,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",43.652270,-79.383516
2,Indigo,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",43.653515,-79.380696
3,CF Toronto Eaton Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",43.653594,-79.380611
4,Eggspectation Bell Trinity Square,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",43.653144,-79.381980


Here we check to make sure Foursquare returned 100 venues

In [19]:
print('{} venues were returned by Foursquare.'.format(venuesNearBy.shape[0]))

100 venues were returned by Foursquare.


#### Explore Neighborhoods in Toronto

Here we create a function to repeat the same process to all the neighborhoods in Toronto

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            clientID, 
            clientSecret, 
            version, 
            tlat, 
            tlong, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called torontoVenues.

In [21]:
torontoVenues=getNearbyVenues(names=torontoData['Neighbourhood'],
                                   latitudes=torontoData['Latitude'],
                                   longitudes=torontoData['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [22]:
print(torontoVenues.shape)
torontoVenues.head()

(3800, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,The Beaches,43.676357,-79.293031,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,The Beaches,43.676357,-79.293031,Indigo,43.653515,-79.380696,Bookstore
3,The Beaches,43.676357,-79.293031,CF Toronto Eaton Centre,43.653594,-79.380611,Shopping Mall
4,The Beaches,43.676357,-79.293031,Eggspectation Bell Trinity Square,43.653144,-79.381980,Breakfast Spot


In [23]:
torontoVenues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton,Exhibition Place,Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",100,100,100,100,100,100
"Cabbagetown,St. James Town",100,100,100,100,100,100
Central Bay Street,100,100,100,100,100,100
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100


Here we find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(torontoVenues['Venue Category'].unique())))

There are 65 uniques categories.


#### Analyze Each Neighborhood

In [26]:
# one hot encoding
torontoOnehot=pd.get_dummies(torontoVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torontoOnehot['Neighborhood'] = torontoVenues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [torontoOnehot.columns[-1]] + list(torontoOnehot.columns[:-1])
torontoOnehot=torontoOnehot[fixed_columns]

torontoOnehot.head()

,Women's Store,Accessories Store,American Restaurant,Asian Restaurant,Bakery,Bank,Bar,Beer Bar,Bookstore,Breakfast Spot,...,Smoothie Shop,Speakeasy,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [27]:
torontoOnehot.shape

(3800, 65)

Here we group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
torontoGrouped=torontoOnehot.groupby('Neighborhood').mean().reset_index()
torontoGrouped.head()

,Neighborhood,Women's Store,Accessories Store,American Restaurant,Asian Restaurant,Bakery,Bank,Bar,Beer Bar,Bookstore,...,Smoothie Shop,Speakeasy,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant
0,"Adelaide,King,Richmond",0.01,0.01,0.04,0.02,0.03,0.01,0.01,0.01,0.01,...,0.01,0.01,0.02,0.03,0.02,0.01,0.03,0.02,0.01,0.01
1,Berczy Park,0.01,0.01,0.04,0.02,0.03,0.01,0.01,0.01,0.01,...,0.01,0.01,0.02,0.03,0.02,0.01,0.03,0.02,0.01,0.01
2,"Brockton,Exhibition Place,Parkdale Village",0.01,0.01,0.04,0.02,0.03,0.01,0.01,0.01,0.01,...,0.01,0.01,0.02,0.03,0.02,0.01,0.03,0.02,0.01,0.01
3,Business Reply Mail Processing Centre 969 Eastern,0.01,0.01,0.04,0.02,0.03,0.01,0.01,0.01,0.01,...,0.01,0.01,0.02,0.03,0.02,0.01,0.03,0.02,0.01,0.01
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.01,0.01,0.04,0.02,0.03,0.01,0.01,0.01,0.01,...,0.01,0.01,0.02,0.03,0.02,0.01,0.03,0.02,0.01,0.01


In [29]:
torontoGrouped.shape

(38, 65)

Here we print each neighborhood along with the top 5 most common venues

In [30]:
num_top_venues = 5

for hood in torontoGrouped['Neighborhood']:
    print("----"+hood+"----")
    temp = torontoGrouped[torontoGrouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0       Clothing Store  0.06
1  American Restaurant  0.04
2             Tea Room  0.03
3           Steakhouse  0.03
4          Coffee Shop  0.03


----Berczy Park----
                 venue  freq
0       Clothing Store  0.06
1  American Restaurant  0.04
2             Tea Room  0.03
3           Steakhouse  0.03
4          Coffee Shop  0.03


----Brockton,Exhibition Place,Parkdale Village----
                 venue  freq
0       Clothing Store  0.06
1  American Restaurant  0.04
2             Tea Room  0.03
3           Steakhouse  0.03
4          Coffee Shop  0.03


----Business Reply Mail Processing Centre 969 Eastern----
                 venue  freq
0       Clothing Store  0.06
1  American Restaurant  0.04
2             Tea Room  0.03
3           Steakhouse  0.03
4          Coffee Shop  0.03


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                 venue  fre

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now lets create the new dataframe and display the top 10 venues for ech neighbourhood

In [33]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted=pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood']=torontoGrouped['Neighborhood']

for ind in np.arange(torontoGrouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:]=return_most_common_venues(torontoGrouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
1,Berczy Park,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
2,"Brockton,Exhibition Place,Parkdale Village",Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
3,Business Reply Mail Processing Centre 969 Eastern,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
5,"Cabbagetown,St. James Town",Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
6,Central Bay Street,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
7,"Chinatown,Grange Park,Kensington Market",Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
8,Christie,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
9,Church and Wellesley,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub


### Cluster Neighborhoods

In [34]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

torontoGroupedClustering=torontoGrouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans=KMeans(n_clusters=kclusters, random_state=0).fit(torontoGroupedClustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Here we create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood

In [35]:
torontoMerged=torontoData
torontoMerged.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)

# add clustering labels
torontoMerged['Cluster Labels']=kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
torontoMerged=torontoMerged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

torontoMerged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub


let's visualize the resulting clusters

In [36]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters=folium.Map(location=[tlat,tlong], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torontoMerged['Latitude'],torontoMerged['Longitude'],torontoMerged['Neighborhood'],torontoMerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

### Cluster 1

In [38]:
torontoMerged.loc[torontoMerged['Cluster Labels'] == 0,torontoMerged.columns[[1] + list(range(5,torontoMerged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
1,East Toronto,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
2,East Toronto,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
3,East Toronto,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
4,Central Toronto,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
5,Central Toronto,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
6,Central Toronto,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
7,Central Toronto,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
8,Central Toronto,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub
9,Central Toronto,0,Clothing Store,American Restaurant,Tea Room,Hotel,Steakhouse,Bakery,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Gastropub


### Cluster 2

In [39]:
torontoMerged.loc[torontoMerged['Cluster Labels'] == 1,torontoMerged.columns[[1] + list(range(5,torontoMerged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


### Cluster 3

In [40]:
torontoMerged.loc[torontoMerged['Cluster Labels'] == 2,torontoMerged.columns[[1] + list(range(5,torontoMerged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


### Cluster 4

In [41]:
torontoMerged.loc[torontoMerged['Cluster Labels'] == 3,torontoMerged.columns[[1] + list(range(5,torontoMerged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


### Cluster 5

In [42]:
torontoMerged.loc[torontoMerged['Cluster Labels'] == 4,torontoMerged.columns[[1] + list(range(5,torontoMerged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
